Now, we turn to elliptic equations. One interpretation of elliptic equations is as steady-state versions of parabolic equations. In other words, the solution to an elliptic equation describes the equilibrium state of a system, rather than how it evolves over time. As an example, recall the forced heat equation:
$$
    u_t + \Delta u = f.
$$
Along with some initial values and boundary conditions, we can model the evolution of the system over time. It is natural to wonder if we can model what happens as time goes to infinity. Since this is a dissipative system, it is reasonable to expect that it approaches some limiting solution as time goes to infinity (contrast this to the wave equation with reflective boundary conditions, in which we would not expect things to converge as time goes to infinity). One way to impose the requirement that eventually, the solution does not change in time is to specify that $u_t = 0$. If we do this, we get the equation
$$
    \Delta u = f.
$$
This is Poisson's equation, and we have heuristically justified the interpretation of it as giving the limiting or steady-state solution to the heat equation. 